### Changing to the main directory

### Import Necessary Libraries

In [1]:
from datasets import load_from_disk
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer

from utils import MODEL_ID, BASE_MODEL_PATH, TRAINING_PATH, PROCESSED_DATA_DIR, FINAL_MODEL_PATH
from utils import EPOCHS, LR, BATCH_SIZE, SAVE_TOTAL_LIMIT, EVALUATION_STRATEGY

import torch
from utils import clear_gpu_memory
import os

### Loading Tokenized Datasets

In [2]:
train_tokenized_data = load_from_disk(os.path.join(PROCESSED_DATA_DIR, "train_tokenized_data"))
val_tokenized_data = load_from_disk(os.path.join(PROCESSED_DATA_DIR, "val_tokenized_data"))
test_tokenized_data = load_from_disk(os.path.join(PROCESSED_DATA_DIR, "test_tokenized_data"))

In [3]:
train_tokenized_data

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 4974
})

In [4]:
val_tokenized_data

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1067
})

In [5]:
test_tokenized_data

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1067
})

### Loading Model and Tokenizer

In [6]:
## Downloading Model 
original_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID, torch_dtype=torch.float32)
## Saving Model
original_model.save_pretrained(BASE_MODEL_PATH)

## Freeing GPU Memory
del original_model
clear_gpu_memory()

c:\Krish- GenAI\FineTuning-nvidiaqa\nvidia_qa_ft\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
loaded_original_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL_PATH, torch_dtype=torch.float32)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
clear_gpu_memory()

c:\Krish- GenAI\FineTuning-nvidiaqa\nvidia_qa_ft\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


We need to use loaded model so not deleting it from the memory.

### Full Fine Tuning

In [8]:
training_args = TrainingArguments(
    output_dir=TRAINING_PATH,
    save_total_limit=SAVE_TOTAL_LIMIT,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,
    num_train_epochs=EPOCHS,
    evaluation_strategy=EVALUATION_STRATEGY,
)

In [9]:
trainer = Trainer(
    model=loaded_original_model,
    args=training_args,
    train_dataset=train_tokenized_data,
    eval_dataset=val_tokenized_data
)

Note: For faster results, do try permutation and combinations of the hyperparameters.

In [ ]:
trainer.train()

In [18]:
## Saving the Model
trainer.model.save_pretrained(FINAL_MODEL_PATH)
tokenizer.save_pretrained(FINAL_MODEL_PATH)

('models/final_model/tokenizer_config.json',
 'models/final_model/special_tokens_map.json',
 'models/final_model/tokenizer.json')

### Clearing Cache Memory from GPU

In [15]:
del loaded_original_model
clear_gpu_memory()